##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ferramenta de autoração do TF Lite

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/guide/authoring"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/lite/guide/authoring.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/lite/guide/authoring.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/lite/guide/authoring.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

A API TensorFlow Lite Authoring (Criação do TensorFlow Lite) oferece uma maneira de manter seus modelos do `tf.function` compatíveis com o TensorFlow Lite.


## Configuração

In [ ]:
import tensorflow as tf

## Problema de compatibilidade entre TensorFlow e TensorFlow Lite

Se você quiser usar seu modelo do TF em dispositivos, precisa convertê-lo em um modelo do TF Lite para usá-lo no interpretador do TF Lite. Durante a conversão, pode ocorrer um erro de compatibilidade devido a operações do TensorFlow incompatíveis com o conjunto de operações integradas do TF Lite.

Esse tipo de erro é bem irritante. Como podemos detectá-lo antecipadamente, como no momento de criação do modelo?

Observe que haverá falha do código abaixo na chamada `converter.convert()`.


In [ ]:
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
result = f(tf.constant([0.0]))
print (f"result = {result}")

In [ ]:
# Convert the tf.function
converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [f.get_concrete_function()], f)
try:
  fb_model = converter.convert()
except Exception as e:
  print(f"Got an exception: {e}")

## Uso básico de autoração para verificar a compatibilidade do alvo

Lançamos a API de criação para detectar problemas de compatibilidade com o TensorFlow lite no momento de criação do modelo.

Basta adicionar o decorador `@tf.lite.experimental.authoring.compatible` para encapsular seu modelo de `tf.function` para verificar a compatibilidade com o TF Lite.

Dessa forma, a compatibilidade será verificada automaticamente quando você avaliar o modelo.

In [ ]:
@tf.lite.experimental.authoring.compatible
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
result = f(tf.constant([0.0]))
print (f"result = {result}")


Se for encontrado algum problema de compatibilidade com o TensorFlow Lite, será exibida a mensagem `COMPATIBILITY WARNING` ou `COMPATIBILITY ERROR` juntamente com o local exato da operação problemática. Neste exemplo, é exibido o local da operação `tf.Cosh` em seu modelo de tf.function.

Além disso, você pode verificar o log de compatibilidade pelo método `<function_name>.get_compatibility_log()`.

In [ ]:
compatibility_log = '\n'.join(f.get_compatibility_log())
print (f"compatibility_log = {compatibility_log}")

## Gere uma exceção em caso de incompatibilidade

É possível fornecer uma opção para o decorador `@tf.lite.experimental.authoring.compatible`. A opção `raise_exception` gera uma exceção quando você tenta avaliar o modelo decorado.

In [ ]:
@tf.lite.experimental.authoring.compatible(raise_exception=True)
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
try:
  result = f(tf.constant([0.0]))
  print (f"result = {result}")
except Exception as e:
  print(f"Got an exception: {e}")

## Especificação do uso de "Operações do TF específicas"

Se você já conhece o uso de [Select TF ops](https://www.tensorflow.org/lite/guide/ops_select) (Operações do TF específicas), pode indicar à API de criação definindo `converter_target_spec`. É o mesmo objeto [tf.lite.TargetSpec](https://www.tensorflow.org/api_docs/python/tf/lite/TargetSpec) que você usará na API [tf.lite.TFLiteConverter](https://www.tensorflow.org/api_docs/python/tf/lite/TFLiteConverter).


In [ ]:
target_spec = tf.lite.TargetSpec()
target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]
@tf.lite.experimental.authoring.compatible(converter_target_spec=target_spec, raise_exception=True)
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
result = f(tf.constant([0.0]))
print (f"result = {result}")


## Verificando a compatibilidade com GPU

Se você deseja garantir que o modelo seja compatível com [delegado de GPU](https://www.tensorflow.org/lite/performance/gpu) do TensorFlow Lite, pode definir `experimental_supported_backends` de [tf.lite.TargetSpec](https://www.tensorflow.org/api_docs/python/tf/lite/TargetSpec).

O exemplo abaixo mostra como garantir a compatibilidade com delegado de GPU do seu modelo. Observe que esse modelo tem problemas de compatibilidade, pois usa um tensor bidimensional com a operação tf.slice e a operação incompatível tf.cosh. Serão exibidas duas mensagens `COMPATIBILITY WARNING` com as informações do local.

In [ ]:
target_spec = tf.lite.TargetSpec()
target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]
target_spec.experimental_supported_backends = ["GPU"]
@tf.lite.experimental.authoring.compatible(converter_target_spec=target_spec)
@tf.function(input_signature=[
    tf.TensorSpec(shape=[4, 4], dtype=tf.float32)
])
def func(x):
  y = tf.cosh(x)
  return y + tf.slice(x, [1, 1], [1, 1])

result = func(tf.ones(shape=(4,4), dtype=tf.float32))

## Saiba mais

Confira mais informações em:

- Documentação da API [tf.function](https://www.tensorflow.org/api_docs/python/tf/function)
- [Melhor desempenho com tf.function](https://www.tensorflow.org/guide/function)
- [Conversor do TensorFlow Lite](https://www.tensorflow.org/lite/models/convert)
- [Analisador de modelos do TensorFlow Lite](https://www.tensorflow.org/lite/guide/model_analyzer)
- [Delegado de GPU do TensorFlow Lite](https://www.tensorflow.org/lite/performance/gpu)